In [1]:
import pandas as pd

In [2]:
path = "/data/kyungbok/FakeAVCeleb_v1.2/meta_data_added.csv"
df = pd.read_csv(path)
df.head()

,source,target1,target2,method,category,type,race,gender,path,Unnamed: 9
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...


In [3]:
def get_subject(row):
    method_column_mapping = {
        "wav2lip": "source",
        "rtvc": "source",
        # "faceswap": "target1",
        "fsgan": "target1",
        "faceswap-wav2lip": "target1",
        "fsgan-wav2lip": "target1",
        "real": "source",
        "freevc": "source",
        "shifted": "source",
    }
    if row["method"] != "faceswap":
        return row[method_column_mapping[row["method"]]]
    else:
        if "id" in row["target1"]:
            return row["target1"]
        else:
            return row["source"]
        

In [5]:
from dataset_utils import split_new_dataset, split_new_dataset_subjects
# train, val, test = split_new_dataset(df)
train, val, test = split_new_dataset_subjects(df)

loading dataset
1400 200 500


In [39]:
import os
test_subset = "all"

root = "/data/kyungbok/FakeAVCeleb_v1.2/"
train_fold = "train_2.txt"
train_fold = os.path.join(root, train_fold)

with open(train_fold) as train_f:
    train_ids = []
    for train_line in train_f:
        train_id = train_line[:7]
        if "id" in train_id:
            train_ids.append(train_id)

val_source = train_ids[:50]
train_source = train_ids[50:]

test_source = df[~df["source"].isin(train_ids)]["source"].unique()
print(len(test_source))

# df["subject"] = df.apply(lambda row: row[method_column_mapping[row["method"]]], axis=1)
df["subject"] = df.apply(get_subject, axis=1)

train_df = df[df["subject"].isin(train_source)]
val_df = df[df["subject"].isin(val_source)]
test_df = df[df["subject"].isin(test_source)]


# # split the dataset into train and test
# # 100
test_source = df["source"].unique()[400:]

# # 50
val_source = df["source"].unique()[350:400]

# # 350
train_source = df["source"].unique()[:350]
# train df
# train_df = df[df["subject"].isin(train_source)]
train_df_A = train_df[train_df["category"] == "A"]
train_df_B = train_df[train_df["category"] == "B"]
train_df_C = (
    train_df[(train_df["category"] == "C") & (train_df["method"] != "faceswap")]
    .groupby("method")
    .sample(n=175, random_state=42)
)
train_df_D = (
    train_df[(train_df["category"] == "D") & (train_df["method"] != "faceswap-wav2lip")]
    .groupby("method")
    .sample(n=175, random_state=42)
)
final_train_df = pd.concat([train_df_A, train_df_B, train_df_C, train_df_D])

# val_df = df[df["subject"].isin(val_source)]
val_df_A = val_df[val_df["category"] == "A"]
val_df_B = val_df[val_df["category"] == "B"]
val_df_C = (
    val_df[(val_df["category"] == "C") & (val_df["method"] != "faceswap")]
    .groupby("method")
    .sample(n=25, random_state=42)
)
val_df_D = (
    val_df[(val_df["category"] == "D") & (val_df["method"] != "faceswap-wav2lip")]
    .groupby("method")
    .sample(n=25, random_state=42)
)

final_val_df = pd.concat([val_df_A, val_df_B, val_df_C, val_df_D])
# test_df = df[df["subject"].isin(test_source)]
test_df_A = test_df[test_df["category"] == "A"]

if test_subset == "all":
    test_df_B = test_df[test_df["category"] == "B"]
    test_df_C = test_df[(test_df["category"] == "C") & (test_df["method"] == "faceswap")].sample(
        n=100, random_state=42
    )
    test_df_D = test_df[(test_df["category"] == "D") & (test_df["method"] == "faceswap-wav2lip")].sample(
        n=100, random_state=42
    )
    test_df_E = test_df[test_df["category"] == "E"].sample(n=100, random_state=42)
    test_df_F = test_df[test_df["category"] == "F"].sample(n=100, random_state=42)
    final_test_df = pd.concat([test_df_A, test_df_B, test_df_C, test_df_D, test_df_E, test_df_F])
else:
    if test_subset == "C":
        test_subset = test_df[(test_df["category"] == "C") & (test_df["method"] == "faceswap")].sample(
            n=100, random_state=42
        )
    elif test_subset == "D":
        test_subset = test_df[
            (test_df["category"] == "D") & (test_df["method"] == "faceswap-wav2lip")
        ].sample(n=100, random_state=42)
    else:
        test_subset = test_df[test_df["category"] == test_subset].sample(n=100, random_state=42)
    final_test_df = pd.concat([test_df_A, test_subset])
print(len(final_train_df), len(final_val_df), len(final_test_df))

100
137
1400 200 600


In [40]:
faceswap = df[df["method"] == "faceswap"]

In [41]:
faceswap[faceswap["target1"] == "0.mp4"]

,source,target1,target2,method,category,type,race,gender,path,Unnamed: 9,subject
1015,id00475,0.mp4,-,faceswap,C,FakeVideo-RealAudio,African,men,00099_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/African/men/id...,id00475
1017,id00476,0.mp4,-,faceswap,C,FakeVideo-RealAudio,African,men,00109_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/African/men/id...,id00476
1020,id00478,0.mp4,-,faceswap,C,FakeVideo-RealAudio,African,men,00206_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/African/men/id...,id00478
1022,id00518,0.mp4,-,faceswap,C,FakeVideo-RealAudio,African,men,00031_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/African/men/id...,id00518
1027,id00761,0.mp4,-,faceswap,C,FakeVideo-RealAudio,African,men,00072_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/African/men/id...,id00761
...,...,...,...,...,...,...,...,...,...,...,...
1706,id06752,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Asian (South),women,00221_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Asian (South)/...,id06752
1710,id07008,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Asian (South),women,00175_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Asian (South)/...,id07008
1714,id07078,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Asian (South),women,00405_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Asian (South)/...,id07078
1718,id07136,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Asian (South),women,00052_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Asian (South)/...,id07136


In [42]:
faceswap[faceswap["path"] == "00028_0.mp4"]

,source,target1,target2,method,category,type,race,gender,path,Unnamed: 9,subject
1190,id01168,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (American),men,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Ame...,id01168
1201,id01201,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (American),men,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Ame...,id01201
1235,id00100,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (American),women,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Ame...,id00100
1307,id00740,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Asian (East),men,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Asian (East)/m...,id00740
1420,id00186,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (European),men,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Eur...,id00186
1453,id00554,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (European),men,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Eur...,id00554
1464,id00963,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (European),men,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Eur...,id00963
1493,id00042,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (European),women,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Eur...,id00042
1505,id00262,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (European),women,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Eur...,id00262
1509,id00271,0.mp4,-,faceswap,C,FakeVideo-RealAudio,Caucasian (European),women,00028_0.mp4,FakeAVCeleb/FakeVideo-RealAudio/Caucasian (Eur...,id00271
